In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from scipy import spatial

/Users/wknq25/opt/anaconda3/envs/geocoder_py38/lib/python3.8/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/var/folders/fg/_n1n9v7x7x1bk7m6gwfkv97m0000gp/T/ipykernel_4715/2438506497.py:2: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to S

In [2]:
year = 1851
country = "EW"
geom_name = "gb1900"
geom_path = f"output/{year}/{country}/{geom_name}/{geom_name}_{country}_{year}.geojson"


new_uid =  (str(geom_name)
+ "_"
+ str(country)
+ "_"
+ str(year)
)
# print(new_uid)

    
processed_geom_data = gpd.read_file(geom_path, driver = "GeoJSON")
processed_geom_data



,gb1900_EW_1851,pin_id,final_text,conparid_51-91,CEN_1851,geometry
0,52d73146c4931d000e00b112_12480_3610002,52d73146c4931d000e00b112,VICARAGE,12480,3610002,POINT (320949.215 328423.258)
1,52d731e3c4931d000800a2a0_12480_3610002,52d731e3c4931d000800a2a0,POLICE STATION,12480,3610002,POINT (320974.573 328029.188)
2,53f75f10e678ef000b00024c_12480_3610002,53f75f10e678ef000b00024c,TY-NEWYDD,12480,3610002,POINT (320859.559 327007.848)
3,52d73131c4931d000500b249_12480_3610002,52d73131c4931d000500b249,SMITHY,12480,3610002,POINT (320959.605 328837.618)
4,52d7316fc4931d000e00b11d_12480_3610002,52d7316fc4931d000e00b11d,WHITE LION INN,12480,3610002,POINT (320992.414 328280.522)
...,...,...,...,...,...,...
1148810,58278e122c66dcdc11012d4c_7091_4100001,58278e122c66dcdc11012d4c,PILLAR,7091,4100001,POINT (465396.739 299670.842)
1148811,58278e1b2c66dcdc11012d54_7091_4100001,58278e1b2c66dcdc11012d54,STATUE,7091,4100001,POINT (465380.701 299619.869)
1148812,58278e252c66dcdc11012d5e_7091_4100001,58278e252c66dcdc11012d5e,STRETTON HALL,7091,4100001,POINT (465352.035 299589.030)
1148813,58278e2e2c66dcdc11012d66_7091_4100001,58278e2e2c66dcdc11012d66,STATUE,7091,4100001,POINT (465376.126 299526.990)


In [179]:
def calc_dist(coords):
    mean_dist = spatial.distance.pdist(np.array(list(zip(coords.x, coords.y)))).mean()
    return mean_dist

def dedup_streets(gdf, dedup_fields, uid_field, max_points, max_dist_btwn_points, ):


    gdf["count"] = gdf.groupby(dedup_fields).transform("size")

    gdf = gdf[gdf["count"] <= max_points].copy()

   
    dist_grouped = gdf.groupby(by=dedup_fields)["geometry"].apply(lambda x: calc_dist(x)).reset_index(name="dist_calc")
    print(dist_grouped)
    gdf_final = pd.merge(left = gdf, right = dist_grouped, on = dedup_fields)
    print(gdf_final)
    gdf_final = gdf_final[(gdf_final["dist_calc"] <= max_dist_btwn_points) | (gdf_final["dist_calc"].isna())]

    gdf_multi_only = gdf_final[gdf_final.duplicated(subset=dedup_fields, keep = False)]


    gdf_multi_only = gdf_multi_only.groupby(dedup_fields)[uid_field].apply(lambda x: str(x.to_list())).reset_index(name = f"{uid_field}_removed")
    gdf_final = pd.merge(left = gdf_final, right = gdf_multi_only, on = dedup_fields, how="left")
    gdf_final = gpd.GeoDataFrame(gdf_final)
    
    return gdf_final

test1 = ["CEN_1851", "conparid_51-91"]
test1.append("final_text")

t = dedup_streets(processed_geom_data.head(3000), test1, "pin_id", 2, 1000)

/Users/wknq25/opt/anaconda3/envs/geocoder_py38/lib/python3.8/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/var/folders/fg/_n1n9v7x7x1bk7m6gwfkv97m0000gp/T/ipykernel_4715/753993408.py:2: RuntimeWarning: Mean of empty slice.
  mean_dist = spatial.distance.pdist(np.array(list(zip(coords.x, coords.y)))).mean()
/Users/wknq25/opt/anaconda3/envs/geocoder_py38/lib/python3.8/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


      CEN_1851  conparid_51-91  \
0       240001             153   
1       240001             153   
2       240001             153   
3       240002             154   
4       240002             154   
...        ...             ...   
2493   3610002           12480   
2494   3610002           12480   
2495   3610002           12480   
2496   3610002           12480   
2497   3610002           12480   

                                             final_text  dist_calc  
0                                      NIGHTINGALE LANE        NaN  
1                                               WAPPING        NaN  
2                                    WAPPING NEW STAIRS        NaN  
3                                            ANN STREET        NaN  
4                                         BARNES STREET        NaN  
...                                                 ...        ...  
2493                                           VICARAGE        NaN  
2494  VYRNWY AQUEDUCT (LIVERPOOL CORPOR

In [180]:
t

,gb1900_EW_1851,pin_id,final_text,conparid_51-91,CEN_1851,geometry,count,dist_calc,pin_id_removed
0,52d73146c4931d000e00b112_12480_3610002,52d73146c4931d000e00b112,VICARAGE,12480,3610002,POINT (320949.215 328423.258),1,NaN,NaN
1,52d731e3c4931d000800a2a0_12480_3610002,52d731e3c4931d000800a2a0,POLICE STATION,12480,3610002,POINT (320974.573 328029.188),1,NaN,NaN
2,53f75f10e678ef000b00024c_12480_3610002,53f75f10e678ef000b00024c,TY-NEWYDD,12480,3610002,POINT (320859.559 327007.848),1,NaN,NaN
3,52d73131c4931d000500b249_12480_3610002,52d73131c4931d000500b249,SMITHY,12480,3610002,POINT (320959.605 328837.618),1,NaN,NaN
4,52d7316fc4931d000e00b11d_12480_3610002,52d7316fc4931d000e00b11d,WHITE LION INN,12480,3610002,POINT (320992.414 328280.522),1,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2523,57e7f0c82c66dc5ef4001fe0_178_350003,57e7f0c82c66dc5ef4001fe0,KING GEORGE STREET,178,350003,POINT (538351.749 177089.592),1,NaN,NaN
2524,57e7ef222c66dc5ef4001ece_178_350003,57e7ef222c66dc5ef4001ece,BLACKHEATH HILL,178,350003,POINT (538037.175 176702.523),1,NaN,NaN
2525,58ed049a2c66dcf8fa0cae95_178_350003,58ed049a2c66dcf8fa0cae95,BRIDGE,178,350003,POINT (538530.952 176778.683),1,NaN,NaN
2526,57e7f0f52c66dc5ef4001ffc_178_350003,57e7f0f52c66dc5ef4001ffc,GLOUCESTER PLACE,178,350003,POINT (538319.555 177286.521),1,NaN,NaN


In [171]:
t[t["count"] == 2]

,CEN_1851,conparid_51-91,final_text,dist_calc,gb1900_EW_1851,pin_id,geometry,count,pin_id_removed
56,250002,159,EAST FERRY ROAD,620.663906,57f540ba2c66dcab3d0151e7_159_250002,57f540ba2c66dcab3d0151e7,POINT (537953.346 178611.181),2,"['57f540ba2c66dcab3d0151e7', '57f5417f2c66dcab..."
57,250002,159,EAST FERRY ROAD,620.663906,57f5417f2c66dcab3d01521b_159_250002,57f5417f2c66dcab3d01521b,POINT (537951.416 179231.842),2,"['57f540ba2c66dcab3d0151e7', '57f5417f2c66dcab..."
64,250002,159,GLENGALL ROAD,427.994163,57f5425c2c66dcab3d015271_159_250002,57f5425c2c66dcab3d015271,POINT (537236.047 179339.218),2,"['57f5425c2c66dcab3d015271', '57f543052c66dcab..."
65,250002,159,GLENGALL ROAD,427.994163,57f543052c66dcab3d0152d0_159_250002,57f543052c66dcab3d0152d0,POINT (537663.807 179353.383),2,"['57f5425c2c66dcab3d015271', '57f543052c66dcab..."
68,250002,159,GRANARY,622.482640,57f540a32c66dcab3d0151e5_159_250002,57f540a32c66dcab3d0151e5,POINT (537669.001 178982.602),2,"['57f540a32c66dcab3d0151e5', '57f543152c66dcab..."
...,...,...,...,...,...,...,...,...,...
2376,3610002,6259,LLIDIART-Y-BWLCH,6.120101,526aadf1dae8a500020049b6_6259_3610002,526aadf1dae8a500020049b6,POINT (322227.974 331626.244),2,"['526a0cce6903920002000407', '526aadf1dae8a500..."
2383,3610002,6259,MAES-TAN-Y-GLWYDEN,282.918134,53f7608de678ef00080001ab_6259_3610002,53f7608de678ef00080001ab,POINT (320603.538 324975.053),2,"['53f7608de678ef00080001ab', '58009b422c66dcab..."
2384,3610002,6259,MAES-TAN-Y-GLWYDEN,282.918134,58009b422c66dcab3d0611e9_6259_3610002,58009b422c66dcab3d0611e9,POINT (320321.565 324998.157),2,"['53f7608de678ef00080001ab', '58009b422c66dcab..."
2425,3610002,6259,PLANHIGFA PRIDDBWLL,217.680966,53fb447e03bc6d0005000145_6259_3610002,53fb447e03bc6d0005000145,POINT (319585.899 325538.699),2,"['53fb447e03bc6d0005000145', '53fb447303bc6d00..."


In [105]:
if type(processed_geom_data) == gpd.GeoDataFrame:
    print("yes")
else:
    print(type(processed_geom_data))


yes


In [65]:
dup_check = t.drop_duplicates(subset=["final_text", "CEN_1851", "conparid_51-91"], keep=False)
dup_check2 = pd.concat([t, dup_check]).drop_duplicates(keep=False)
dup_check2

,CEN_1851,conparid_51-91,final_text,dist_calc,gb1900_EW_1851,pin_id,geometry,count
8,10001,1,BLOMFIELD ROAD,371.620159,586bbc832c66dcbab800009e_1_10001,586bbc832c66dcbab800009e,POINT (526011.983 181896.330),2
9,10001,1,BLOMFIELD ROAD,371.620159,57ed1ed62c66dca3220007d6_1_10001,57ed1ed62c66dca3220007d6,POINT (526357.102 182034.151),2
23,10001,1,CHURCH,132.275240,57ed1f072c66dca322000802_1_10001,57ed1f072c66dca322000802,POINT (526612.531 181855.549),3
24,10001,1,CHURCH,132.275240,57ed1f372c66dca322000824_1_10001,57ed1f372c66dca322000824,POINT (526626.115 181728.862),3
25,10001,1,CHURCH,132.275240,57ed1f032c66dca322000800_1_10001,57ed1f032c66dca322000800,POINT (526564.178 181913.027),3
...,...,...,...,...,...,...,...,...
1010686,6230006,12755,SALTINGS,857.749956,52e75292002407000b008fd5_12755_6230006,52e75292002407000b008fd5,POINT (226145.673 379215.799),3
1010687,6230006,12755,SALTINGS,857.749956,52e752bd002407000e008415_12755_6230006,52e752bd002407000e008415,POINT (226073.794 378934.097),3
1010688,6230006,12755,SALTINGS,857.749956,52e748ee002407000e007830_12755_6230006,52e748ee002407000e007830,POINT (227031.775 378416.223),3
1010701,6230006,12755,STANLEY MILL,30.032936,59760d872c66dc085a14ea4d_12755_6230006,59760d872c66dc085a14ea4d,POINT (226611.612 378835.888),2


In [86]:
trial1 = t[t.duplicated(subset=["final_text", "CEN_1851", "conparid_51-91"], keep = False)]
trial1

,CEN_1851,conparid_51-91,final_text,dist_calc,gb1900_EW_1851,pin_id,geometry,count
8,10001,1,BLOMFIELD ROAD,371.620159,586bbc832c66dcbab800009e_1_10001,586bbc832c66dcbab800009e,POINT (526011.983 181896.330),2
9,10001,1,BLOMFIELD ROAD,371.620159,57ed1ed62c66dca3220007d6_1_10001,57ed1ed62c66dca3220007d6,POINT (526357.102 182034.151),2
23,10001,1,CHURCH,132.275240,57ed1f072c66dca322000802_1_10001,57ed1f072c66dca322000802,POINT (526612.531 181855.549),3
24,10001,1,CHURCH,132.275240,57ed1f372c66dca322000824_1_10001,57ed1f372c66dca322000824,POINT (526626.115 181728.862),3
25,10001,1,CHURCH,132.275240,57ed1f032c66dca322000800_1_10001,57ed1f032c66dca322000800,POINT (526564.178 181913.027),3
...,...,...,...,...,...,...,...,...
1010686,6230006,12755,SALTINGS,857.749956,52e75292002407000b008fd5_12755_6230006,52e75292002407000b008fd5,POINT (226145.673 379215.799),3
1010687,6230006,12755,SALTINGS,857.749956,52e752bd002407000e008415_12755_6230006,52e752bd002407000e008415,POINT (226073.794 378934.097),3
1010688,6230006,12755,SALTINGS,857.749956,52e748ee002407000e007830_12755_6230006,52e748ee002407000e007830,POINT (227031.775 378416.223),3
1010701,6230006,12755,STANLEY MILL,30.032936,59760d872c66dc085a14ea4d_12755_6230006,59760d872c66dc085a14ea4d,POINT (226611.612 378835.888),2


In [66]:
testing = dup_check2.groupby(["final_text", "CEN_1851", "conparid_51-91"])["pin_id"].apply(list).reset_index(name = "pin_id_removed")
testing

,final_text,CEN_1851,conparid_51-91,pin_id_removed
0,(BERKSHIRE),1230003,1897,"[580242942c66dcab3d06ea68, 580248e72c66dcab3d0..."
1,(BUCKINGHAMSHIRE),1500003,1688,"[580fc8612c66dc1ce5043648, 580fc8872c66dc1ce50..."
2,(DEVON),3150001,5333,"[583afda72c66dcd5d00097e3, 583b02df2c66dcd5d00..."
3,(HEYWOOD & MIDDLETON WATER BOARD),4760006,9085,"[5805493d2c66dcab3d087698, 580546712c66dcab3d0..."
4,(HUNTINGDONSHIRE),1740001,2287,"[581e5a7a2c66dc2d4202142e, 581e5b772c66dc2d420..."
...,...,...,...,...
33589,YSTRADFELLTE,5840003,11917,"[526e7971962d3600020066b5, 526e7986cf836300020..."
33590,YSTRADOWEN,5830002,11869,"[528e914430219e2aec00005d, 528e91be30219e2aec0..."
33591,ZENNOR,3110002,5274,"[57e152f52c66dcf84f00059a, 57fe12f52c66dcab3d0..."
33592,ZION CHAP.,5820002,11854,"[52af40effd3236196b00013b, 5279b0c3b5d44900050..."


In [68]:
testing[testing["final_text"] == "SALTINGS"].to_csv("quick2.tsv",sep="\t")

In [69]:
quick = pd.merge(left = t, right = testing, on = ["final_text", "CEN_1851", "conparid_51-91"], how="left")
quick

,CEN_1851,conparid_51-91,final_text,dist_calc,gb1900_EW_1851,pin_id,geometry,count,pin_id_removed
0,10001,1,ALDRIDGE ROAD VILLAS,0.0,5866e7a42c66dc10b806942d_1_10001,5866e7a42c66dc10b806942d,POINT (524926.445 181641.475),1,NaN
1,10001,1,ALEXANDER STREET,0.0,5866eb6f2c66dc10b806956c_1_10001,5866eb6f2c66dc10b806956c,POINT (525500.238 181417.898),1,NaN
2,10001,1,ALFRED ROAD,0.0,5866e7e42c66dc10b8069443_1_10001,5866e7e42c66dc10b8069443,POINT (525340.084 181780.915),1,NaN
3,10001,1,AMBERLEY ROAD,0.0,586bbc4d2c66dc10b807dd40_1_10001,586bbc4d2c66dc10b807dd40,POINT (525510.673 182020.598),1,NaN
4,10001,1,ARTESIAN ROAD,0.0,5867ad832c66dc10b806af87_1_10001,5867ad832c66dc10b806af87,POINT (525095.940 181170.202),1,NaN
...,...,...,...,...,...,...,...,...,...
924146,6230006,12755,WILLIAM STREET,0.0,5917c6de2c66dc085a00a462_12755_6230006,5917c6de2c66dc085a00a462,POINT (224586.042 382497.392),1,NaN
924147,6230006,12755,YNYS-Y-GÔF,0.0,5917ca992c66dc085a00a4e4_12755_6230006,5917ca992c66dc085a00a4e4,POINT (226253.807 378843.221),1,NaN
924148,6230006,12755,YR-EFAIL,0.0,52f6b2bb80141d000d006a12_12755_6230006,52f6b2bb80141d000d006a12,POINT (226666.335 378017.958),1,NaN
924149,6230006,12755,YSGOLDY-CWERYD,0.0,530d9045553a19000c000370_12755_6230006,530d9045553a19000c000370,POINT (224966.986 380948.218),1,NaN


In [63]:
quick2 = quick[quick["count"]> 1].sample(20)
quick2

,CEN_1851,conparid_51-91,final_text,dist_calc,gb1900_EW_1851,pin_id,geometry,count,pin_id_removed
217784,2320002,3637,TOWING PATH,754.901204,5880d1b82c66dc67e206b1e3_3637_2320002,5880d1b82c66dc67e206b1e3,POINT (625811.699 320514.780),2,"[5880d25a2c66dc67e206b260, 5880d1b82c66dc67e20..."
647661,5000003,9627,BRICK WORKS,815.220916,584d6a652c66dc0ad402ae24_9627_5000003,584d6a652c66dc0ad402ae24,POINT (432576.043 434433.254),2,"[584d6a652c66dc0ad402ae24, 580944e42c66dc1ce50..."
190218,2080002,2963,HENHAM'S FARM,5.915256,5851e0532c66dc0ad40428b5_2963_2080002,5851e0532c66dc0ad40428b5,POINT (580593.443 225693.206),2,"[5851e0532c66dc0ad40428b5, 59d113212c66dc72d00..."
495781,4440002,8267,WINDMILL (PUMPING),23.961744,585428322c66dc10b8006535_8267_4440002,585428322c66dc10b8006535,POINT (435830.582 329303.755),2,"[585428322c66dc10b8006535, 58d952612c66dcf8fa0..."
713589,5320005,10507,INCLINE,85.984415,593ef72c2c66dc085a09ebcf_10507_5320005,593ef72c2c66dc085a09ebcf,POINT (468559.911 508854.505),2,"[593ef72c2c66dc085a09ebcf, 59ba796f2c66dc085a1..."
20355,440002,266,CLAY PIT,541.372437,587e4bdb2c66dc67e2055683_266_440002,587e4bdb2c66dc67e2055683,POINT (529656.471 150460.310),2,"[587e4bdb2c66dc67e2055683, 587e4b862c66dc67e20..."
128571,1450002,1635,ROUSEBARN LANE,722.244152,5857d15a2c66dc10b8019536_1635_1450002,5857d15a2c66dc10b8019536,POINT (508310.452 196748.297),3,"[5841c02f2c66dc145f000599, 5841be002c66dc145f0..."
144435,1620001,1993,SPRING,956.583870,57eae65c2c66dcd15d005168_1993_1620001,57eae65c2c66dcd15d005168,POINT (427838.603 217278.472),2,"[57eae56a2c66dcd15d0050a7, 57eae65c2c66dcd15d0..."
444370,4040002,6880,BOAT HO.,487.372309,5977853b2c66dc085a1525c7_6880_4040002,5977853b2c66dc085a1525c7,POINT (422722.093 257170.045),2,"[5823a0a52c66dc2d420453ea, 5977853b2c66dc085a1..."
470453,4290001,7640,OLD GRAVEL PIT,686.581225,57f389bd2c66dcab3d007c35_7640_4290001,57f389bd2c66dcab3d007c35,POINT (518675.667 379208.281),3,"[57f389bd2c66dcab3d007c35, 57f389de2c66dcab3d0..."


In [70]:
quick.to_csv("quick2.tsv",sep="\t")

In [78]:
quick

,CEN_1851,conparid_51-91,final_text,dist_calc,gb1900_EW_1851,pin_id,geometry,count,pin_id_removed
0,10001,1,ALDRIDGE ROAD VILLAS,0.0,5866e7a42c66dc10b806942d_1_10001,5866e7a42c66dc10b806942d,POINT (524926.445 181641.475),1,NaN
1,10001,1,ALEXANDER STREET,0.0,5866eb6f2c66dc10b806956c_1_10001,5866eb6f2c66dc10b806956c,POINT (525500.238 181417.898),1,NaN
2,10001,1,ALFRED ROAD,0.0,5866e7e42c66dc10b8069443_1_10001,5866e7e42c66dc10b8069443,POINT (525340.084 181780.915),1,NaN
3,10001,1,AMBERLEY ROAD,0.0,586bbc4d2c66dc10b807dd40_1_10001,586bbc4d2c66dc10b807dd40,POINT (525510.673 182020.598),1,NaN
4,10001,1,ARTESIAN ROAD,0.0,5867ad832c66dc10b806af87_1_10001,5867ad832c66dc10b806af87,POINT (525095.940 181170.202),1,NaN
...,...,...,...,...,...,...,...,...,...
924146,6230006,12755,WILLIAM STREET,0.0,5917c6de2c66dc085a00a462_12755_6230006,5917c6de2c66dc085a00a462,POINT (224586.042 382497.392),1,NaN
924147,6230006,12755,YNYS-Y-GÔF,0.0,5917ca992c66dc085a00a4e4_12755_6230006,5917ca992c66dc085a00a4e4,POINT (226253.807 378843.221),1,NaN
924148,6230006,12755,YR-EFAIL,0.0,52f6b2bb80141d000d006a12_12755_6230006,52f6b2bb80141d000d006a12,POINT (226666.335 378017.958),1,NaN
924149,6230006,12755,YSGOLDY-CWERYD,0.0,530d9045553a19000c000370_12755_6230006,530d9045553a19000c000370,POINT (224966.986 380948.218),1,NaN


In [79]:
quick["length"] = quick[quick["pin_id_removed"].isna() == False].apply(lambda x: len(x["pin_id_removed"]), axis=1)
quick

,CEN_1851,conparid_51-91,final_text,dist_calc,gb1900_EW_1851,pin_id,geometry,count,pin_id_removed,length
0,10001,1,ALDRIDGE ROAD VILLAS,0.0,5866e7a42c66dc10b806942d_1_10001,5866e7a42c66dc10b806942d,POINT (524926.445 181641.475),1,NaN,NaN
1,10001,1,ALEXANDER STREET,0.0,5866eb6f2c66dc10b806956c_1_10001,5866eb6f2c66dc10b806956c,POINT (525500.238 181417.898),1,NaN,NaN
2,10001,1,ALFRED ROAD,0.0,5866e7e42c66dc10b8069443_1_10001,5866e7e42c66dc10b8069443,POINT (525340.084 181780.915),1,NaN,NaN
3,10001,1,AMBERLEY ROAD,0.0,586bbc4d2c66dc10b807dd40_1_10001,586bbc4d2c66dc10b807dd40,POINT (525510.673 182020.598),1,NaN,NaN
4,10001,1,ARTESIAN ROAD,0.0,5867ad832c66dc10b806af87_1_10001,5867ad832c66dc10b806af87,POINT (525095.940 181170.202),1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
924146,6230006,12755,WILLIAM STREET,0.0,5917c6de2c66dc085a00a462_12755_6230006,5917c6de2c66dc085a00a462,POINT (224586.042 382497.392),1,NaN,NaN
924147,6230006,12755,YNYS-Y-GÔF,0.0,5917ca992c66dc085a00a4e4_12755_6230006,5917ca992c66dc085a00a4e4,POINT (226253.807 378843.221),1,NaN,NaN
924148,6230006,12755,YR-EFAIL,0.0,52f6b2bb80141d000d006a12_12755_6230006,52f6b2bb80141d000d006a12,POINT (226666.335 378017.958),1,NaN,NaN
924149,6230006,12755,YSGOLDY-CWERYD,0.0,530d9045553a19000c000370_12755_6230006,530d9045553a19000c000370,POINT (224966.986 380948.218),1,NaN,NaN


In [80]:
quick.groupby(["count","length"]).size()

count  length
2      2.0       60332
3      3.0       10284
dtype: int64

In [127]:

dedup_fields = ["CEN_1851", "conparid_51-91"]
dedup_fields.append("final_text")
dedup_fields

['CEN_1851', 'conparid_51-91', 'final_text']

In [129]:
quick[quick["count"]==2]

,CEN_1851,conparid_51-91,final_text,dist_calc,gb1900_EW_1851,pin_id,geometry,count,pin_id_removed,length
8,10001,1,BLOMFIELD ROAD,371.620159,586bbc832c66dcbab800009e_1_10001,586bbc832c66dcbab800009e,POINT (526011.983 181896.330),2,"[586bbc832c66dcbab800009e, 57ed1ed62c66dca3220...",2.0
9,10001,1,BLOMFIELD ROAD,371.620159,57ed1ed62c66dca3220007d6_1_10001,57ed1ed62c66dca3220007d6,POINT (526357.102 182034.151),2,"[586bbc832c66dcbab800009e, 57ed1ed62c66dca3220...",2.0
55,10001,1,HARROW ROAD,929.421985,5866e8292c66dc10b8069453_1_10001,5866e8292c66dc10b8069453,POINT (525617.334 181712.736),2,"[5866e8292c66dc10b8069453, 57ed1f2f2c66dca3220...",2.0
56,10001,1,HARROW ROAD,929.421985,57ed1f2f2c66dca32200081c_1_10001,57ed1f2f2c66dca32200081c,POINT (526543.536 181635.433),2,"[5866e8292c66dc10b8069453, 57ed1f2f2c66dca3220...",2.0
141,10002,1,CLEVELAND SQUARE,153.856266,5866e9602c66dc10b80694b0_1_10002,5866e9602c66dc10b80694b0,POINT (526235.889 181157.529),2,"[5866e9602c66dc10b80694b0, 5866e96c2c66dc10b80...",2.0
...,...,...,...,...,...,...,...,...,...,...
924007,6230006,12755,LLAIN-FAIN,3.297397,597ef09b2c66dc085a163550_12755_6230006,597ef09b2c66dc085a163550,POINT (223167.053 382738.877),2,"[530d8dc2553a190008000311, 597ef09b2c66dc085a1...",2.0
924029,6230006,12755,MONUMENT,2.145508,528cbf47aa240c000d00585f_12755_6230006,528cbf47aa240c000d00585f,POINT (225082.617 382242.788),2,"[528cbf47aa240c000d00585f, 5917c5002c66dc085a0...",2.0
924030,6230006,12755,MONUMENT,2.145508,5917c5002c66dc085a00a41d_12755_6230006,5917c5002c66dc085a00a41d,POINT (225080.473 382242.867),2,"[528cbf47aa240c000d00585f, 5917c5002c66dc085a0...",2.0
924102,6230006,12755,STANLEY MILL,30.032936,59760d872c66dc085a14ea4d_12755_6230006,59760d872c66dc085a14ea4d,POINT (226611.612 378835.888),2,"[59760d872c66dc085a14ea4d, 52d6e80cc4931d00080...",2.0


In [130]:
quick['liststring'] = [','.join(map(str, l)) for l in quick['pin_id_removed']]

TypeError: 'float' object is not iterable

In [146]:
t["testing"] = [str(x) for x in t["pin_id_removed"]]
# quick[quick["count"]==2]

In [148]:
t2 = t.drop(columns=["pin_id_removed"])

In [172]:
t.to_file("quick_test.geojson")

In [ ]:
# Notes
# If a label appears 3 or more times then it's probably not a street and should be removed

# How do you calculate the distance between all points with the same name

# Remove labels where there are 3 or more duplicates within a jurisdiction; this leaves two; calculate distance between them; if less than 1km keep one; 